### Preparing Data ###



In [ ]:
import pandas as pd
from scipy.spatial import cKDTree
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Хакатон/dataset2015_2019.csv')

In [125]:
data.head(2)

,Unnamed: 0,centroid,x,y,RotationLength,RotationCount,CODE_CULTU_2015,CODE_CULTU_2016,CODE_CULTU_2017,CODE_CULTU_2018,CODE_CULTU_2019,CODE_GROUP_2015,CODE_GROUP_2016,CODE_GROUP_2017,CODE_GROUP_2018,CODE_GROUP_2019,cultuCode,groupCode,hzs,kg_id,ff_id,1 Nearest Neigbour Point,2 Nearest Neigbour Point,3 Nearest Neigbour Point,4 Nearest Neigbour Point
0,0,Point (907753.82834152 6554634.46488151),907753.828342,6.554634e+06,1,5,PPH,PPH,PPH,PPH,PPH,18,18,18,18,18,aaaaa,aaaaa,7b,Dfb,ff12,Point (907748.69100314 6554212.06490037),Point (907410.62452802 6554909.79074054),Point (908089.34400838 6554953.05455597),Point (907563.65755459 6554039.35762193)
1,1,Point (906932.34642625 6550537.14396784),906932.346426,6.550537e+06,1,5,PPH,PPH,PPH,PPH,PPH,18,18,18,18,18,aaaaa,aaaaa,7b,Dfb,ff12,Point (906980.7207283 6550304.97315954),Point (906736.32315476 6550335.52813463),Point (906595.72670333 6550598.70621165),Point (907032.95563938 6550121.6369531)


In [ ]:
!pip install catboost

     |████████████████████████████████| 65.7MB 79kB/s 


In [ ]:
import catboost as cb
from catboost import Pool, CatBoostClassifier
from sklearn.model_selection import GridSearchCV

In [10]:
def predict_next_culture_4_years_list(l0, l1, l2, l3):
  if l0 == l1 != l2 and l2 == l3:
     return 'Unknown'
  if l0 == l1 == l2 == l3:
     return l0
  #if l0 == l3:
  #   return l1
  if l0 == l2 and l1 == l3:
     return l0
  #if l0 == l2 and l1 != l2:
  #   return l0
  if l0 == l1 != l2 and l2 == l3:
     return l0
  return 'Unknown'

In [11]:
print(predict_next_culture_4_years_list(data.iloc[0]['CODE_CULTU_2015'],data.iloc[0]['CODE_CULTU_2016'],data.iloc[0]['CODE_CULTU_2017'],data.iloc[0]['CODE_CULTU_2018']))

PPH


In [12]:
g = data.groupby(['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018'])['CODE_CULTU_2019'].count()

In [13]:
gm = g.add_suffix('').reset_index()

In [14]:
gm_uniq = gm[gm['CODE_CULTU_2019'] == 1]

In [15]:
gm_uniq

,CODE_CULTU_2015,CODE_CULTU_2016,CODE_CULTU_2017,CODE_CULTU_2018,CODE_CULTU_2019
0,ACA,FLA,FLA,FLA,1
1,ACA,FLP,FLP,FLP,1
2,ACA,MIS,MIS,SNE,1
3,ACA,SNE,SNE,SNE,1
5,AGR,BOR,SNE,SNE,1
...,...,...,...,...,...
139698,VRT,VRT,VRT,SOG,1
139703,XFE,PTR,PTR,BTH,1
139704,XFE,PTR,PTR,PPH,1
139706,XFE,XFE,XFE,PRL,1


In [16]:
gm_pred = pd.merge(gm_uniq, data,  how='left', left_on=['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018'], right_on = ['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018'])


In [17]:
gm_pred = gm_pred.reindex(columns=['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018','CODE_CULTU_2019_y'])

In [19]:
data_pred = pd.merge(data, gm_pred,  how='left', left_on=['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018'], right_on = ['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018'])

In [20]:
data_pred['CODE_CULTU_2019_y'].value_counts()

BTH    11247
PTR     8134
LUZ     4978
MLG     4003
MIS     3902
       ...  
LFP        1
MIN        1
OEH        1
PSY        1
MRG        1
Name: CODE_CULTU_2019_y, Length: 218, dtype: int64

In [21]:
data_part1 = data_pred[data_pred.CODE_CULTU_2019_y.notnull()].copy()

In [22]:
data_part1.head(2)

,Unnamed: 0,centroid,x,y,RotationLength,RotationCount,CODE_CULTU_2015,CODE_CULTU_2016,CODE_CULTU_2017,CODE_CULTU_2018,CODE_CULTU_2019,CODE_GROUP_2015,CODE_GROUP_2016,CODE_GROUP_2017,CODE_GROUP_2018,CODE_GROUP_2019,cultuCode,groupCode,hzs,kg_id,ff_id,1 Nearest Neigbour Point,2 Nearest Neigbour Point,3 Nearest Neigbour Point,4 Nearest Neigbour Point,CODE_CULTU_2019_y
50,50,Point (859900.11098517 6591026.78278985),859900.110985,6.591027e+06,3,1,BTH,MID,MIS,MIS,BTH,1,2,2,2,1,abcca,abbba,8b,Cfa,ff13,Point (859812.49760217 6591185.50103266),Point (859719.0802948 6590975.56193138),Point (860081.26015362 6591098.77150968),Point (860019.18559373 6590736.93947271),BTH
74,74,Point (848022.43324874 6589869.38021483),848022.433249,6.589869e+06,4,1,AVP,SGH,MIS,MIS,ORP,4,4,2,2,3,abccd,aabbc,8b,Cfa,ff13,Point (848060.93345233 6589936.96049742),Point (847937.44700784 6589858.58590304),Point (848069.49079389 6590019.03797348),Point (848120.55312173 6589719.68664306),ORP


In [24]:
data_part2 = data_pred[data_pred.CODE_CULTU_2019_y.isna()].copy()

In [25]:
data_part2.head(2)

,Unnamed: 0,centroid,x,y,RotationLength,RotationCount,CODE_CULTU_2015,CODE_CULTU_2016,CODE_CULTU_2017,CODE_CULTU_2018,CODE_CULTU_2019,CODE_GROUP_2015,CODE_GROUP_2016,CODE_GROUP_2017,CODE_GROUP_2018,CODE_GROUP_2019,cultuCode,groupCode,hzs,kg_id,ff_id,1 Nearest Neigbour Point,2 Nearest Neigbour Point,3 Nearest Neigbour Point,4 Nearest Neigbour Point,CODE_CULTU_2019_y
0,0,Point (907753.82834152 6554634.46488151),907753.828342,6.554634e+06,1,5,PPH,PPH,PPH,PPH,PPH,18,18,18,18,18,aaaaa,aaaaa,7b,Dfb,ff12,Point (907748.69100314 6554212.06490037),Point (907410.62452802 6554909.79074054),Point (908089.34400838 6554953.05455597),Point (907563.65755459 6554039.35762193),NaN
1,1,Point (906932.34642625 6550537.14396784),906932.346426,6.550537e+06,1,5,PPH,PPH,PPH,PPH,PPH,18,18,18,18,18,aaaaa,aaaaa,7b,Dfb,ff12,Point (906980.7207283 6550304.97315954),Point (906736.32315476 6550335.52813463),Point (906595.72670333 6550598.70621165),Point (907032.95563938 6550121.6369531),NaN


In [26]:
data_part2['CODE_CULTU_2019_y'] = data_part2.apply(lambda x: predict_next_culture_4_years_list(x['CODE_CULTU_2015'],x['CODE_CULTU_2016'],x['CODE_CULTU_2017'],x['CODE_CULTU_2018']), axis=1, result_type='expand')

In [27]:
data_part2['CODE_CULTU_2019_y'].value_counts()

Unknown    1453574
PPH        1067851
VRC         181807
J6S         100032
SNE          85849
            ...   
LIP              2
NVF              2
SAD              2
CZP              2
LDP              2
Name: CODE_CULTU_2019_y, Length: 160, dtype: int64

In [28]:
data_manual = data_part2[data_part2['CODE_CULTU_2019_y'] != 'Unknown']

In [29]:
data_manual.shape

(2078679, 26)

In [30]:
Y_val = data_manual['CODE_CULTU_2019']
Y_pred = data_manual['CODE_CULTU_2019_y']

In [31]:
print(accuracy_score(Y_val,Y_pred))

0.9479198086861896


In [32]:
data_part3 = data_part2[data_part2['CODE_CULTU_2019_y'] == 'Unknown']

In [33]:
print(data_part1.shape)
print(data_part2.shape)
print(data_part3.shape)

(82551, 26)
(3532253, 26)
(1453574, 26)


### Train/Val CatBoost###

In [35]:
"""
'RotationLength', 'RotationCount',
'CODE_CULTU_2015', 'CODE_CULTU_2016', 'CODE_CULTU_2017',
'CODE_CULTU_2018', 'CODE_CULTU_2019', 'CODE_GROUP_2015',
'CODE_GROUP_2016', 'CODE_GROUP_2017', 'CODE_GROUP_2018',
'CODE_GROUP_2019', 'cultuCode', 'groupCode', 'hzs', 'kg_id', 'ff_id'
"""

X = data_part3[['RotationLength', 'RotationCount',
       'CODE_CULTU_2015', 'CODE_CULTU_2016', 'CODE_CULTU_2017',
       'CODE_CULTU_2018', 'CODE_GROUP_2015',
       'CODE_GROUP_2016', 'CODE_GROUP_2017', 'CODE_GROUP_2018',
       'cultuCode', 'groupCode', 'hzs', 'kg_id', 'ff_id']]

X = X.replace(np.nan, 'unknown', regex=True)

Y = data_part3['CODE_CULTU_2019']

In [ ]:
X.shape

(1453574, 15)

In [36]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=286)

In [37]:
X.columns

Index(['RotationLength', 'RotationCount', 'CODE_CULTU_2015', 'CODE_CULTU_2016',
       'CODE_CULTU_2017', 'CODE_CULTU_2018', 'CODE_GROUP_2015',
       'CODE_GROUP_2016', 'CODE_GROUP_2017', 'CODE_GROUP_2018', 'cultuCode',
       'groupCode', 'hzs', 'kg_id', 'ff_id'],
      dtype='object')

In [38]:
X.head(2)

,RotationLength,RotationCount,CODE_CULTU_2015,CODE_CULTU_2016,CODE_CULTU_2017,CODE_CULTU_2018,CODE_GROUP_2015,CODE_GROUP_2016,CODE_GROUP_2017,CODE_GROUP_2018,cultuCode,groupCode,hzs,kg_id,ff_id
12,3,1,TTH,ORH,TTH,PTR,4,3,4,19,abacc,abacc,8b,Cfb,ff13
18,3,1,ORH,ORH,TTH,SGH,3,3,4,4,aabcb,aabbb,8b,Cfb,ff13


In [39]:
cat_features = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [40]:
X.isnull().sum(axis = 0)

RotationLength     0
RotationCount      0
CODE_CULTU_2015    0
CODE_CULTU_2016    0
CODE_CULTU_2017    0
CODE_CULTU_2018    0
CODE_GROUP_2015    0
CODE_GROUP_2016    0
CODE_GROUP_2017    0
CODE_GROUP_2018    0
cultuCode          0
groupCode          0
hzs                0
kg_id              0
ff_id              0
dtype: int64

In [41]:
train_dataset = Pool(data=X_train,
                     label=Y_train,
                     cat_features=cat_features)

eval_dataset = Pool(data=X_val,
                    label=Y_val,
                    cat_features=cat_features)

# Initialize CatBoostClassifier
model = CatBoostClassifier(iterations=100,
                           #learning_rate=0.25,
                           depth=4,
                           loss_function='MultiClass',  task_type='GPU')

#model = CatBoostClassifier(loss_function='MultiClass',  task_type='GPU')

# Fit model
model.fit(train_dataset)  
# Get predicted classes
preds_class = model.predict(eval_dataset)
# Get predicted probabilities for each class
preds_proba = model.predict_proba(eval_dataset)
# Get predicted RawFormulaVal
preds_raw = model.predict(eval_dataset, prediction_type='RawFormulaVal')

Learning rate set to 0.5
0:	learn: 13.9392360	total: 6.81s	remaining: 11m 13s
1:	learn: 2140.9018428	total: 14.1s	remaining: 11m 33s
2:	learn: 5977.3900567	total: 21s	remaining: 11m 18s
3:	learn: 6921.1037417	total: 27.8s	remaining: 11m 7s
4:	learn: 7777.9509158	total: 34.6s	remaining: 10m 58s
5:	learn: 7458.7790540	total: 41.5s	remaining: 10m 49s
6:	learn: 6919.1510372	total: 48.3s	remaining: 10m 41s
7:	learn: 6588.2039491	total: 55.1s	remaining: 10m 34s
8:	learn: 5837.8934265	total: 1m 1s	remaining: 10m 26s
9:	learn: 5566.6427383	total: 1m 8s	remaining: 10m 19s
10:	learn: 5262.7261878	total: 1m 15s	remaining: 10m 11s
11:	learn: 4851.5535452	total: 1m 22s	remaining: 10m 4s
12:	learn: 4746.1858746	total: 1m 29s	remaining: 9m 57s
13:	learn: 4646.1730614	total: 1m 36s	remaining: 9m 50s
14:	learn: 4494.6242597	total: 1m 42s	remaining: 9m 43s
15:	learn: 4378.0731181	total: 1m 49s	remaining: 9m 36s
16:	learn: 4098.0306366	total: 1m 56s	remaining: 9m 29s
17:	learn: 3951.0186256	total: 2m 3s	

In [42]:
print(accuracy_score(Y_val,preds_class))

0.6135080749187348


In [43]:
eval_dataset = Pool(data=X,
                    label=Y,
                    cat_features=cat_features)

In [44]:
preds_class = model.predict(eval_dataset)

In [45]:
print(accuracy_score(Y,preds_class))

0.6139625502382403


In [46]:
data_part3['CODE_CULTU_2019_y'] = preds_class

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [47]:
data_part2_2 = data_part2[data_part2['CODE_CULTU_2019_y'] != 'Unknown']

final_data = pd.concat([data_part1,data_part2_2,data_part3])

In [48]:
Y_val = final_data['CODE_CULTU_2019']
Y_pred = final_data['CODE_CULTU_2019_y']

In [49]:
print(accuracy_score(Y_val,Y_pred))

0.8148192820412946


In [ ]:
###Корректировка явных ошибок CatBoost### (Предсказание значения, которое не встречалось в выборке, заменяем на последнее значение в выборке)

In [50]:
data_part3_1 = data_part3.loc[(data_part3['CODE_CULTU_2019_y'] != data_part3['CODE_CULTU_2015']) & (data_part3['CODE_CULTU_2019_y'] != data_part3['CODE_CULTU_2016']) & (data_part3['CODE_CULTU_2019_y'] != data_part3['CODE_CULTU_2017']) & (data_part3['CODE_CULTU_2019_y'] != data_part3['CODE_CULTU_2018']) ]

In [51]:
data_part3_2 = data_part3.loc[(data_part3['CODE_CULTU_2019_y'] == data_part3['CODE_CULTU_2015']) | (data_part3['CODE_CULTU_2019_y'] == data_part3['CODE_CULTU_2016']) | (data_part3['CODE_CULTU_2019_y'] == data_part3['CODE_CULTU_2017']) | (data_part3['CODE_CULTU_2019_y'] == data_part3['CODE_CULTU_2018']) ]

In [52]:
data_part3_1['CODE_CULTU_2019_y'] = data_part3_1['CODE_CULTU_2018']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [54]:
final_data = pd.concat([data_part1,data_part2_2,data_part3_1,data_part3_2])

In [55]:
Y_val = final_data['CODE_CULTU_2019']
Y_pred = final_data['CODE_CULTU_2019_y']

In [56]:
print(accuracy_score(Y_val,Y_pred))

0.8190662619605378


##Full CatBoost###

In [57]:
#X2 = data_part3[['RotationLength', 'RotationCount',
#       'CODE_CULTU_2015', 'CODE_CULTU_2016', 'CODE_CULTU_2017',
#       'CODE_CULTU_2018', 'CODE_GROUP_2015',
#       'CODE_GROUP_2016', 'CODE_GROUP_2017', 'CODE_GROUP_2018',
#       'cultuCode', 'groupCode', 'hzs', 'kg_id', 'ff_id']]

X2 = data_part3[['RotationLength', 'RotationCount',
       'CODE_CULTU_2015', 'CODE_CULTU_2016', 'CODE_CULTU_2017',
       'CODE_CULTU_2018', 'CODE_GROUP_2015',
       'CODE_GROUP_2016', 'CODE_GROUP_2017', 'CODE_GROUP_2018',
       'hzs', 'kg_id', 'ff_id']]


X2 = X2.replace(np.nan, 'unknown', regex=True)

Y2 = data_part3['CODE_CULTU_2019']

In [58]:
cat_features2 = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [59]:
train_dataset2 = Pool(data=X2,
                     label=Y2,
                     cat_features=cat_features2)

eval_dataset2 = Pool(data=X2,
                    label=Y2,
                    cat_features=cat_features2)

# Initialize CatBoostClassifier
model2 = CatBoostClassifier(iterations=200,
                           #learning_rate=0.25,
                           depth=6,
                           loss_function='MultiClass',  task_type='GPU')

#model = CatBoostClassifier(loss_function='MultiClass',  task_type='GPU')

# Fit model
model2.fit(train_dataset2)  
# Get predicted classes
preds_class2 = model2.predict(eval_dataset2)
# Get predicted probabilities for each class
preds_proba2 = model2.predict_proba(eval_dataset2)
# Get predicted RawFormulaVal
preds_raw2 = model2.predict(eval_dataset2, prediction_type='RawFormulaVal')

Learning rate set to 0.5
0:	learn: 14.2548560	total: 8.8s	remaining: 29m 11s
1:	learn: 1157.3928097	total: 18.8s	remaining: 31m 1s
2:	learn: 2730.9936584	total: 27.8s	remaining: 30m 24s
3:	learn: 3467.8979784	total: 36.7s	remaining: 29m 58s
4:	learn: 3646.6619463	total: 45.6s	remaining: 29m 38s
5:	learn: 3703.5599151	total: 54.4s	remaining: 29m 20s
6:	learn: 3297.7898394	total: 1m 3s	remaining: 29m 5s
7:	learn: 3188.8005041	total: 1m 12s	remaining: 28m 51s
8:	learn: 2659.5263592	total: 1m 21s	remaining: 28m 40s
9:	learn: 2452.2489478	total: 1m 30s	remaining: 28m 30s
10:	learn: 2207.6678862	total: 1m 38s	remaining: 28m 20s
11:	learn: 2118.3410270	total: 1m 47s	remaining: 28m 9s
12:	learn: 2051.8803996	total: 1m 56s	remaining: 27m 59s
13:	learn: 1965.7683916	total: 2m 5s	remaining: 27m 50s
14:	learn: 1904.6344417	total: 2m 14s	remaining: 27m 40s
15:	learn: 1842.2370241	total: 2m 23s	remaining: 27m 31s
16:	learn: 1825.9514273	total: 2m 32s	remaining: 27m 21s
17:	learn: 1777.6758252	total:

In [60]:
print(accuracy_score(Y2,preds_class2))

0.572658839522446


In [61]:
data_part3['CODE_CULTU_2019_y'] = preds_class

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [62]:
data_part2_2 = data_part2[data_part2['CODE_CULTU_2019_y'] != 'Unknown']

In [63]:
Y_val = final_data['CODE_CULTU_2019']
Y_pred = final_data['CODE_CULTU_2019_y']

In [64]:
print(accuracy_score(Y_val,Y_pred))

0.8190662619605378


In [ ]:
###Корректировка явных ошибок CatBoost### (Предсказание значения, которое не встречалось в выборке, заменяем на последнее значение в выборке)

In [65]:
data_part3_1 = data_part3.loc[(data_part3['CODE_CULTU_2019_y'] != data_part3['CODE_CULTU_2015']) & (data_part3['CODE_CULTU_2019_y'] != data_part3['CODE_CULTU_2016']) & (data_part3['CODE_CULTU_2019_y'] != data_part3['CODE_CULTU_2017']) & (data_part3['CODE_CULTU_2019_y'] != data_part3['CODE_CULTU_2018']) ]

In [66]:
data_part3_2 = data_part3.loc[(data_part3['CODE_CULTU_2019_y'] == data_part3['CODE_CULTU_2015']) | (data_part3['CODE_CULTU_2019_y'] == data_part3['CODE_CULTU_2016']) | (data_part3['CODE_CULTU_2019_y'] == data_part3['CODE_CULTU_2017']) | (data_part3['CODE_CULTU_2019_y'] == data_part3['CODE_CULTU_2018']) ]

In [67]:
data_part3_1['CODE_CULTU_2019_y'] = data_part3_1['CODE_CULTU_2018']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [68]:
final_data2 = pd.concat([data_part1,data_part2_2,data_part3_1,data_part3_2])

In [69]:
Y_val = final_data2['CODE_CULTU_2019']
Y_pred = final_data2['CODE_CULTU_2019_y']

In [70]:
print(accuracy_score(Y_val,Y_pred))

0.8190662619605378


## TEST Predict ##

In [71]:
test2015 = pd.read_csv('/content/drive/MyDrive/Хакатон/test_2015.csv')

In [72]:
test2016 = pd.read_csv('/content/drive/MyDrive/Хакатон/test_2016.csv')

In [73]:
test2017 = pd.read_csv('/content/drive/MyDrive/Хакатон/test_2017.csv')

In [74]:
test2018 = pd.read_csv('/content/drive/MyDrive/Хакатон/test_2018.csv')

In [75]:
test2015_2018 = pd.merge(test2015, test2016,  how='left', left_on=['centroid'], right_on = ['centroid'])

In [76]:
test2015_2018

,CODE_CULTU_x,CODE_GROUP_x,centroid,CODE_CULTU_y,CODE_GROUP_y
0,PPH,18,Point (882657.94895066 6569342.10998129),PPH,18
1,PPH,18,Point (882115.97784605 6572755.67171272),PPH,18
2,TTH,4,Point (881577.18757373 6569639.00274748),ORH,3
3,PPH,18,Point (861602.67865301 6594153.2889364),PPH,18
4,MIS,2,Point (860126.47223916 6591185.82080882),BTH,1
...,...,...,...,...,...
401640,PPH,18,Point (626983.84393005 6887703.12019497),PPH,18
401641,SBO,28,Point (635405.32546667 6883716.73243333),SBO,28
401642,SNE,28,Point (658006.88162677 6881175.08919206),J5M,11
401643,SNE,28,Point (653022.3464582 6888260.28248648),SNE,28


In [77]:
test2015_2018.rename(columns = {'CODE_CULTU_x': 'CODE_CULTU_2015', 'CODE_GROUP_x': 'CODE_GROUP_2015', 'CODE_CULTU_y': 'CODE_CULTU_2016', 'CODE_GROUP_y': 'CODE_GROUP_2016'}, inplace = True)

In [78]:
test2015_2018 = pd.merge(test2015_2018, test2017,  how='left', left_on=['centroid'], right_on = ['centroid'])

In [79]:
test2015_2018.rename(columns = {'CODE_CULTU': 'CODE_CULTU_2017', 'CODE_GROUP': 'CODE_GROUP_2017'}, inplace = True)

In [80]:
test2015_2018 = pd.merge(test2015_2018, test2018,  how='left', left_on=['centroid'], right_on = ['centroid'])

In [81]:
test2015_2018.rename(columns = {'CODE_CULTU': 'CODE_CULTU_2018', 'CODE_GROUP': 'CODE_GROUP_2018'}, inplace = True)

In [82]:
test2015_2018.head(2)

,CODE_CULTU_2015,CODE_GROUP_2015,centroid,CODE_CULTU_2016,CODE_GROUP_2016,CODE_CULTU_2017,CODE_GROUP_2017,CODE_CULTU_2018,CODE_GROUP_2018
0,PPH,18,Point (882657.94895066 6569342.10998129),PPH,18,PPH,18,PPH,18
1,PPH,18,Point (882115.97784605 6572755.67171272),PPH,18,PPH,18,PPH,18


In [86]:
test2015_2018.head(2)

,CODE_CULTU_2015,CODE_GROUP_2015,centroid,CODE_CULTU_2016,CODE_GROUP_2016,CODE_CULTU_2017,CODE_GROUP_2017,CODE_CULTU_2018,CODE_GROUP_2018
0,PPH,18,Point (882657.94895066 6569342.10998129),PPH,18,PPH,18,PPH,18
1,PPH,18,Point (882115.97784605 6572755.67171272),PPH,18,PPH,18,PPH,18


In [87]:
test2015_2018_phz_kg_ff = pd.read_csv('/content/drive/MyDrive/Хакатон/test2015-2018_phz_kg_ff.csv')

In [88]:
test2015_2018_phz_kg_ff

,centroid,hzs,kg_id,ff_id
0,Point (837113.21815327 6542642.03294466),8b,Cfb,ff13
1,Point (859951.6897682 6548701.23025595),8b,Cfb,ff13
2,Point (869152.69888415 6554341.41204209),8b,Cfb,ff13
3,Point (843914.62975223 6570993.98124897),8b,Cfb,ff13
4,Point (842372.94899233 6570245.72791538),8b,Cfb,ff13
...,...,...,...,...
401490,Point (464629.98946606 6207835.49881565),6a,ET,ff5
401491,Point (464742.14669457 6210914.3467607),6a,ET,ff5
401492,Point (467190.57989509 6200166.76301246),5b,ET,ff5
401493,Point (467425.07238401 6200107.01837806),5b,ET,ff5


In [89]:
test2015_2018 = pd.merge(test2015_2018, test2015_2018_phz_kg_ff,  how='left', left_on=['centroid'], right_on = ['centroid'])

In [91]:
test2015_2018.head(2)

,CODE_CULTU_2015,CODE_GROUP_2015,centroid,CODE_CULTU_2016,CODE_GROUP_2016,CODE_CULTU_2017,CODE_GROUP_2017,CODE_CULTU_2018,CODE_GROUP_2018,hzs,kg_id,ff_id
0,PPH,18,Point (882657.94895066 6569342.10998129),PPH,18,PPH,18,PPH,18,8b,Cfb,ff13
1,PPH,18,Point (882115.97784605 6572755.67171272),PPH,18,PPH,18,PPH,18,8b,Cfb,ff13


In [92]:
test2015_2018_rot = pd.read_csv('/content/drive/MyDrive/Хакатон/Хакатонtest.csv')

In [93]:
test2015_2018 = pd.merge(test2015_2018, test2015_2018_rot,  how='left', left_on=['centroid'], right_on = ['centroid'])

In [94]:
test2015_2018.head(2)

,CODE_CULTU_2015,CODE_GROUP_2015,centroid,CODE_CULTU_2016,CODE_GROUP_2016,CODE_CULTU_2017,CODE_GROUP_2017,CODE_CULTU_2018,CODE_GROUP_2018,hzs,kg_id,ff_id,2015,2016,2017,2018,RotationLength,RotationCount
0,PPH,18,Point (882657.94895066 6569342.10998129),PPH,18,PPH,18,PPH,18,8b,Cfb,ff13,PPH,PPH,PPH,PPH,1,4
1,PPH,18,Point (882115.97784605 6572755.67171272),PPH,18,PPH,18,PPH,18,8b,Cfb,ff13,PPH,PPH,PPH,PPH,1,4


In [95]:
g_test = test2015_2018.groupby(['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018']).count()

In [96]:
gm_test = g_test.add_suffix('').reset_index()

In [98]:
gm_test.head(2)

,CODE_CULTU_2015,CODE_CULTU_2016,CODE_CULTU_2017,CODE_CULTU_2018,CODE_GROUP_2015,centroid,CODE_GROUP_2016,CODE_GROUP_2017,CODE_GROUP_2018,hzs,kg_id,ff_id,2015,2016,2017,2018,RotationLength,RotationCount
0,ACA,PPH,PPH,PPH,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,AGR,AGR,AGR,AGR,5,5,5,5,5,2,2,2,5,5,5,5,5,5


In [99]:
gm_pred_g2 = gm_pred.drop_duplicates(['CODE_CULTU_2019_y'])

In [100]:
gm_pred_g2.head(2)

,CODE_CULTU_2015,CODE_CULTU_2016,CODE_CULTU_2017,CODE_CULTU_2018,CODE_CULTU_2019_y
0,ACA,FLA,FLA,FLA,FLA
1,ACA,FLP,FLP,FLP,FLP


In [101]:
gm_test_pred = pd.merge(gm_test, gm_pred,  how='left', left_on=['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018'], right_on = ['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018'])


In [102]:
gm_test_pred = gm_test_pred.reindex(columns=['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018','CODE_CULTU_2019_y'])

In [103]:
gm_test_pred['CODE_CULTU_2019_y'].value_counts()

BTH    683
PTR    444
LUZ    250
MLG    219
MIS    215
      ... 
AGR      1
PAG      1
PWT      1
CRD      1
PAT      1
Name: CODE_CULTU_2019_y, Length: 144, dtype: int64

In [104]:
test_pred = pd.merge(test2015_2018, gm_test_pred,  how='left', left_on=['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018'], right_on = ['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018'])

In [105]:
test_pred['CODE_CULTU_2019_y'].value_counts()

BTH    735
PTR    462
LUZ    263
MLG    244
MIS    225
      ... 
EST      1
PAT      1
FNO      1
LEF      1
TCR      1
Name: CODE_CULTU_2019_y, Length: 144, dtype: int64

In [106]:
test_part1 = test_pred[test_pred.CODE_CULTU_2019_y.notnull()].copy()

In [107]:
test_part1.head(2)

,CODE_CULTU_2015,CODE_GROUP_2015,centroid,CODE_CULTU_2016,CODE_GROUP_2016,CODE_CULTU_2017,CODE_GROUP_2017,CODE_CULTU_2018,CODE_GROUP_2018,hzs,kg_id,ff_id,2015,2016,2017,2018,RotationLength,RotationCount,CODE_CULTU_2019_y
290,MLG,16,Point (904995.49612915 6521680.81797805),MIE,2,MPC,8,MIE,2,8b,Dfc,ff13,MLG,MIE,MPC,MIE,3,1,MCR
372,ORH,3,Point (897295.88784029 6521430.88754548),TTH,4,ORH,3,LU7,16,8a,Dfc,ff13,ORH,TTH,ORH,LU7,3,1,LUZ


In [108]:
test_part2 = test_pred[test_pred.CODE_CULTU_2019_y.isna()].copy()

In [109]:
test_part2['CODE_CULTU_2019_y'] = test_part2.apply(lambda x: predict_next_culture_4_years_list(x['CODE_CULTU_2015'],x['CODE_CULTU_2016'],x['CODE_CULTU_2017'],x['CODE_CULTU_2018']), axis=1, result_type='expand')

In [110]:
test_part2['CODE_CULTU_2019_y'].value_counts()

Unknown    166549
PPH        118597
VRC         19986
J6S         11119
SNE          9443
            ...  
RDF             1
FFO             1
TRD             1
CCT             1
SA5             1
Name: CODE_CULTU_2019_y, Length: 145, dtype: int64

In [111]:
test_part3 = test_part2[test_part2['CODE_CULTU_2019_y'] == 'Unknown']

In [112]:
test_part3.head(2)

,CODE_CULTU_2015,CODE_GROUP_2015,centroid,CODE_CULTU_2016,CODE_GROUP_2016,CODE_CULTU_2017,CODE_GROUP_2017,CODE_CULTU_2018,CODE_GROUP_2018,hzs,kg_id,ff_id,2015,2016,2017,2018,RotationLength,RotationCount,CODE_CULTU_2019_y
2,TTH,4,Point (881577.18757373 6569639.00274748),ORH,3,TTH,4,SGH,4,8b,Cfb,ff13,TTH,ORH,TTH,SGH,3,1,Unknown
14,MIE,2,Point (937384.1439264 6587125.63904581),BTH,1,BTH,1,MIE,2,8b,Cfb,ff11,MIE,BTH,BTH,MIE,2,2,Unknown


In [113]:
test_part3 = test_part3.add_suffix('').reset_index()

In [114]:
X_test = test_part3[['RotationLength', 'RotationCount',
       'CODE_CULTU_2015', 'CODE_CULTU_2016', 'CODE_CULTU_2017',
       'CODE_CULTU_2018', 'CODE_GROUP_2015',
       'CODE_GROUP_2016', 'CODE_GROUP_2017', 'CODE_GROUP_2018',
       'hzs', 'kg_id', 'ff_id']]


X_test = X_test.replace(np.nan, 'unknown', regex=True)

In [115]:
cat_features_test = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [116]:
test_dataset2 = Pool(data=X_test,
                    cat_features=cat_features_test)


# Get predicted classes
preds_class2 = model2.predict(test_dataset2)
# Get predicted probabilities for each class
#preds_proba2 = model2.predict_proba(eval_dataset2)
# Get predicted RawFormulaVal
#preds_raw2 = model2.predict(eval_dataset2, prediction_type='RawFormulaVal')

In [117]:
test_part3['CODE_CULTU_2019_y'] = preds_class2

In [118]:
test_part2_2 = test_part2[test_part2['CODE_CULTU_2019_y'] != 'Unknown']

In [ ]:
###Корректировка явных ошибок CatBoost### (Предсказание значения, которое не встречалось в выборке, заменяем на последнее значение в выборке)

In [119]:
test_part3_1 = test_part3.loc[(test_part3['CODE_CULTU_2019_y'] != test_part3['CODE_CULTU_2015']) & (test_part3['CODE_CULTU_2019_y'] != test_part3['CODE_CULTU_2016']) & (test_part3['CODE_CULTU_2019_y'] != test_part3['CODE_CULTU_2017']) & (test_part3['CODE_CULTU_2019_y'] != test_part3['CODE_CULTU_2018']) ]

In [120]:
test_part3_2 = test_part3.loc[(test_part3['CODE_CULTU_2019_y'] == test_part3['CODE_CULTU_2015']) | (test_part3['CODE_CULTU_2019_y'] == test_part3['CODE_CULTU_2016']) | (test_part3['CODE_CULTU_2019_y'] == test_part3['CODE_CULTU_2017']) | (test_part3['CODE_CULTU_2019_y'] == test_part3['CODE_CULTU_2018']) ]

In [121]:
test_part3_1['CODE_CULTU_2019_y'] = test_part3_1['CODE_CULTU_2018']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [122]:
final_data_test = pd.concat([test_part1,test_part2_2,test_part3_1,test_part3_2])

In [ ]:
pd.DataFrame(final_data_test[['centroid','CODE_CULTU_2019_y']]).to_csv("/content/drive/MyDrive/Хакатон/predict2019.csv",index=False,header=True)